##### Algorithms and Data Structures (Winter - Spring 2022)

* [Colab view](https://colab.research.google.com/github/4dsolutions/elite_school/blob/master/ADS_intro_1.ipynb)
* [nbviewer view](https://nbviewer.org/github/4dsolutions/elite_school/blob/master/ADS_intro_1.ipynb)
* [ACSL / USACO](Exercises.ipynb)
* [Repo](https://github.com/4dsolutions/elite_school/)
* Main Text 
    -   [Algorithms](ADS_intro_1.ipynb) 
    -   [Algorithms (continued)](ADS_intro_2.ipynb)
    -   [Structured Data](ADS_intro_3.ipynb)
* Projects 
    -   [Wordle](ADS_project_1.ipynb)
* Research 
    -   [Word Sets and Graph Theory](ADS_research_1.ipynb)
* Sandboxes 
    -   [Spatial Geometry](ADS_sandbox.ipynb)  
    -   [Finite Groups](ADS_sandbox_2.ipynb)  
    -   [Chess](ADS_sandbox_3.ipynb)
    -   [Soccer](ADS_sandbox_4.ipynb)
    -   [Maps](ADS_sandbox_5.ipynb)
    -   [Graphs](ADS_sandbox_6.ipynb)

*Lets talk more about...*
# Graph Theory

FAQ: 
* Q: What is a graph? <br /> 
  A: Nodes connected by edges.
* Q: Is a polyhedron a graph? <br />A: Yes, by some definitions.
* Q: What terminology applies, pertaining to Graphs? <br /> 
  A: I'm so glad you asked.

What computer scientists often talk about, in connection with Graph Theory, are road and rail networks.  Roads come together at intersections, which are sometimes traffic circles.  Rail roads have switches and round houses, allowing trains to access all the connected tracks, even if they must reverse direction to do so sometimes.

In thinking about road and rail, we also think about direction.  A directed graph tells us in which direction a road runs, perhaps in both, perhaps not. Ditto with railways.  

Perhaps a route is one way north to south for some hours, and south to north at other hours.  Freeways with extra express lanes may change their directionality to coincide with rush hour requirements.

A final factor we will consider is "weight" which can mean a lot of things, such as distance.  

However "weight" may fold in many more measurements than simply "time between nodes" at a certain speed.  

Regarding the "weightiness" of a Route:

* How twisty is this road? 
* Do wild animals suddenly appear in the headlights?
* What about the number of lanes?  
* How stressful is this road to drive?  
* Does this road have any gas stops or other facilities?
* Does this road feature steep drop offs with no guard rails?  

One might imagine a long checklist of criteria, adding up to what we would call a segment's score.  

Once we have a score associated with each edge, we're ready to apply such algorithm's as Djikstra's.

Djikstra's Algorithm is about minizine the total weight between any starting node, and every other node in the network.

#### Representing Graph Data Structures

However, before we start implementing the repeating steps of the Djikstra Algorithm, or other algorithms, we need a way to store and retrieve graph representations.  

How will we tell ourselves what nodes could be next, from where we are now?  

We will need to know what node are reachable from the current node. We may need to remember where we've come from, so we don't hop back.  

If the graph has directionality (some are always both directional along every edge), and/or if it's weighted, how will we register this information for the various algorithms to make sense of?

Let's consider some articles: 

* [Graph Basics Contributed by: Ruchi Nayyar](https://www.mygreatlearning.com/blog/representing-graphs-in-data-structures/)

* [Representing Graph Data Structures by Jackson Gilkey](https://towardsdatascience.com/representing-graph-data-structures-6e3c19ccd677)

* [Weighted Graphs at Emory](https://www.mathcs.emory.edu/~cheung/Courses/171/Syllabus/11-Graph/weighted.html)

A graph can be represented using 3 data structures: 

* adjacency matrix
* adjacency list
* adjacency set

####  Adjacency Matrix

This is one solution for storing both directionality and weighting. Think of a Cartesian product, or just product, pairing every element in a set with every other, including itself.

In [11]:
import numpy as np
import pandas as pd

In [12]:
mult_table = np.product([np.array([1,2,3], dtype=int).reshape(1, 3), 
             np.array([1,2,3], dtype=int).reshape(3,1)])
mult_table

array([[1, 2, 3],
       [2, 4, 6],
       [3, 6, 9]])

In [13]:
caley_table = pd.DataFrame(data = mult_table, 
                           index = ['a','b','c'], 
                           columns=['a','b','c'])
caley_table

,a,b,c
a,1,2,3
b,2,4,6
c,3,6,9


However in an adjacency matrix, we're not looking for a product, but whether the two nodes are connected, yes or no.  

If the idea is "row to column", then a 1 in (a, b) but not in (b, a) would mean traffic could only go in the (a, b) direction -- in which case we have our directionality.  

Then if the nodes k, m are connected, they might have some number stored at W(k, m), not just 1 or 0.  A positive number would be the weight, whereas a 0 would mean the obvious: that a node is connected to only itself.

Therefore an adjacency matrix might look like this, using random lengths, randomly interconnection some nodes.

In [34]:
mult_table = np.random.randint(0, 10, (5, 5))
graph_data = mult_table * np.random.randint(0, 2, (5,5), dtype = int)
graph_data

array([[0, 0, 7, 0, 0],
       [0, 6, 1, 0, 0],
       [0, 8, 3, 8, 3],
       [0, 0, 0, 0, 5],
       [0, 9, 4, 0, 0]])

In [36]:
graph_table = pd.DataFrame(data = graph_data, 
                           index = ['a', 'b', 'c', 'd', 'e'], 
                           columns = ['a','b', 'c', 'd', 'e'], dtype=int)

graph_table

,a,b,c,d,e
a,0,0,7,0,0
b,0,6,1,0,0
c,0,8,3,8,3
d,0,0,0,0,5
e,0,9,4,0,0


####  Adjacency List

An adjacency list might use (node1, node2) tuples, with node1 -> node2 the implied direction.  Store (node2, node1) for a route in the other direction.  Tuples are valid keys in a dictionary.  The values will be the weights of each road.

Once we have selected our workaday data structure for the graph itself, we will need additional structures in order to accomplish Djikstra's algorithm.  

For example, we will need a dictionary associating every node with a shortest distance attained for it so far.  We will also want lists of visted and unvisited nodes.

Suppose we have selected the data structure below to represent our graph.  ('node0', node1'): weight

In [38]:
graph = {('a','b'): 5,
         ('b','a'): 5,
         ('b','c'): 6,
         ('b','d'): 3,
         ('d','e'): 1,
         ('e','d'): 2,
         ('e','a'): 5,
         ('a','e'): 2}

In [45]:
current = 'b'
previous = 'a'
f = filter(lambda pair: (pair[0] == current) and (pair[1] != previous), graph)

In [46]:
list(f)

[('b', 'c'), ('b', 'd')]

In [59]:
def get_next(current: str, previous: str, G: dict) -> list:
    results = []
    f = filter(lambda pair: (pair[0] == current) and (pair[1] != previous), G)
    for e in f:
        results.append((e, G[e]))
    return sorted(results, key=lambda p: p[1])

In [61]:
get_next('b', 'a', graph)

[(('b', 'd'), 3), (('b', 'c'), 6)]

In [62]:
def get_nodes(G:dict) -> list:
    the_nodes = [ ]
    # more code goes here
    return the_nodes

### How Do We Represent Polyhedrons?

The data structure for a polyhedron often starts with faces as a list of nodes.  We go around the face in order, clockwise or counter-clockwise.  Does it matter in which way?  It might.  To be clear, clockwise looking from the outside in is the same as counterclockwise looking from inside out.

Given faces are nodes in a ring, that last node connects the the first.

From faces we may distill edges, e.g. if a face is (a, b, c, d) then the edges used would be (a, b) (b, c), (c, d) and (a, d).  Keeping edge pairs unique may involve keeping them alphabetical e.g. store (a, b) only as (a, b) and not as (b, a). If what we have is a directed graph, this design might change.

The nodes themselves get to be vectors with tails at the origin i.e. if edge (a, b) is a segment between two vectors, then a and b are both vectors with tails at the origin and with tips at points a and b respectively.

With this apparatus in place we're ready to:

* scale a polyhedron (grow and shrink the vectors)
* rotate a polyhedron (rotate each of the vectors)
* write the polyhedron in POV-Ray scene description language

